In [1]:
# we always start with a dataset to train on. let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-04-23 14:06:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.36MB/s    in 0.3s    

2025-04-23 14:06:50 (3.36 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for i,c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])


In [6]:
print(encode('an encoded piece of text'))
decode(encode("hiii there"))

[39, 52, 1, 43, 52, 41, 53, 42, 43, 42, 1, 54, 47, 43, 41, 43, 1, 53, 44, 1, 58, 43, 62, 58]


'hiii there'

In [7]:
import jax
import jax.numpy as jnp
data = jnp.array(encode(text), dtype=jnp.int64)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

/var/folders/lb/d206k2v920v_0_rnh_dm8r3w0000gn/T/ipykernel_40077/3220451585.py:3: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in array is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  data = jnp.array(encode(text), dtype=jnp.int64)


(1115394,) int32
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 43 53 54 50 43  8  0  0 13 50 50 10  0 35 43  1 49 52 53 61  5
 58  6  1 61 43  1 49 52 53 61  5 58  8  0  0 18 47 56 57 58  1 15 47 58
 47 64 43 52 10  0 24 43 58  1 59 57  1 49 47 50 50  1 46 47 51  6  1 39
 52 42  1 61 43  5 50 50  1 46 39 

In [8]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(key, split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # generate random starting positions for the examples
    ix = jax.random.randint(key, (batch_size,), 0, len(data) - block_size)
    x = jnp.stack([data[i:i+block_size] for i in ix])
    y = jnp.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

key = jax.random.PRNGKey(4206969)
xb, yb = get_batch(key, 'train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
(4, 8)
[[19 50 53 59 41 43 57 58]
 [45 43  1 40 43 57 58  1]
 [56 52 39 56 42 47 52 43]
 [61 39 58 41 46 47 52 45]]
targets:
(4, 8)
[[50 53 59 41 43 57 58 43]
 [43  1 40 43 57 58  1 46]
 [52 39 56 42 47 52 43 12]
 [39 58 41 46 47 52 45  1]]
----
when input is [19] the target: 50
when input is [19, 50] the target: 53
when input is [19, 50, 53] the target: 59
when input is [19, 50, 53, 59] the target: 41
when input is [19, 50, 53, 59, 41] the target: 43
when input is [19, 50, 53, 59, 41, 43] the target: 57
when input is [19, 50, 53, 59, 41, 43, 57] the target: 58
when input is [19, 50, 53, 59, 41, 43, 57, 58] the target: 43
when input is [45] the target: 43
when input is [45, 43] the target: 1
when input is [45, 43, 1] the target: 40
when input is [45, 43, 1, 40] the target: 43
when input is [45, 43, 1, 40, 43] the target: 57
when input is [45, 43, 1, 40, 43, 57] the target: 58
when input is [45, 43, 1, 40, 43, 57, 58] the target: 1
when input is [45, 43, 1, 40, 43, 57, 58, 1] th

In [28]:
import flax
import flax.linen as nn
import optax

class BigramLanguageModel(nn.Module):
    num_embeddings: int
    features: int

    @nn.compact
    def __call__(self, idx):
        logits = nn.Embed(self.num_embeddings, self.features)(idx)
        return logits

    def generate(self, key, params, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits = model.apply(params, idx)
            logits = logits[:, -1, :] # (1, C) C=embed dim here
            key, subkey = jax.random.split(key)
            idx_next = jax.random.categorical(subkey, logits, shape=(logits.shape[0],1))
            idx = jnp.concat([idx, idx_next], axis=1).astype(int)
            # e.g.
            # [[ 0 ]]
            # [[ 0 38]]
            # [[ 0 38 42]]
            # [[ 0 38 42 63]]
            # [[ 0 38 42 63 49]]
            # . . .
        return idx

        
model = BigramLanguageModel(num_embeddings=vocab_size, features=vocab_size)

def loss_fn(params, x, y):
    logits = model.apply(params, x) # (B, T, C)
    # y (B, T) array of indexes for next char/token
    # also standard to have ys as one-hot vectors (representing true probability distribution) 
    # then they would have shape (B, T, C)
    # and you would use optax.losses.softmax_cross_entropy(logirs, one_hot_y).mean() instead
    return optax.losses.softmax_cross_entropy_with_integer_labels(logits, y).mean()

init_key, gen_key = jax.random.split(jax.random.PRNGKey(402323))
params = model.init(init_key, jnp.ones_like(xb))
logits = model.apply(params, xb)
loss = loss_fn(params, xb, yb)

print(logits.shape)
print(loss)

print(decode(model.generate(gen_key, params, jnp.zeros((1,1), dtype=jnp.int32), 100)[0].tolist()))

(4, 8, 65)
4.1729555

ZdyksdKDtFNefnwVh3TBvXT:ZiRUDg; Px$$MLfAdcEzfE,V-SeGyXGy tdZijl'?3fqBTrvAaH3PP&vrC3wDQWgl$SSH' sIdv



In [29]:

tx = optax.adam(learning_rate=1e-3)
opt_state = tx.init(params)
loss_grad_fn = jax.value_and_grad(loss_fn)
for i in range(10000):
    init_key, train_key = jax.random.split(init_key)
    xb, yb = get_batch(train_key, 'train')
    loss, grads = loss_grad_fn(params, xb, yb)
    updates, opt_state = tx.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
print(loss)


2.2430437


In [ ]:
print(decode(model.generate(jax.random.PRNGKey(43523523), params, jnp.zeros((1,1), dtype=jnp.int32), 500)[0].tolist()))


bera thwoft l hair ch araner dangundtapre orn at aive allo hece ipr'lpllis w tomed an

THOTougor?-
Mof f ite,

Mouthit, bex;
k; an'd!'Andeno ple, turson tidsorstt, LIVINRI!Dgrg to ond rewifayzllHagndvod thred, civy beat'FPOur akeph minondour ce, bEim merere!

Wee shegodeswd y CAREnge h KAssor wawat to wnporct
GqBlist pou d pou.
ADUTJNash mofoumstcaing ur alyo t y nthee ha imuill s brrd het t ndiny ve' t e an t;Y pllds mar s akTyRif tre theo he
Berefe t d, u? CHEV:
Anis
RAlofe I'd akeingnco

The 


In [25]:
B, T, C = 4, 8, 32
key = jax.random.PRNGKey(34343)
x = jax.random.normal(key, (B,T,C))
tril = jnp.tril(jnp.ones((T,T)))
wei = jnp.zeros((T,T))
wei = wei.at[tril == 0].set(float('-inf')) 
wei = jax.nn.softmax(wei,axis=-1)
print(wei)
out = wei @ x
out[0]

[[1.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.5        0.5        0.         0.         0.         0.
  0.         0.        ]
 [0.33333334 0.33333334 0.33333334 0.         0.         0.
  0.         0.        ]
 [0.25       0.25       0.25       0.25       0.         0.
  0.         0.        ]
 [0.2        0.2        0.2        0.2        0.2        0.
  0.         0.        ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667
  0.         0.        ]
 [0.14285715 0.14285715 0.14285715 0.14285715 0.14285715 0.14285715
  0.14285715 0.        ]
 [0.125      0.125      0.125      0.125      0.125      0.125
  0.125      0.125     ]]


Array([[-9.36835468e-01,  6.82493269e-01, -9.05545354e-01,
         1.27150500e+00, -7.41448328e-02, -3.09238374e-01,
        -6.49880350e-01,  1.67768037e+00, -1.36831605e+00,
         9.45524216e-01, -8.55683327e-01,  7.82965541e-01,
         1.39927375e+00,  9.77689698e-02,  5.22718132e-01,
         1.90649256e-02, -1.25525091e-02,  9.05659974e-01,
        -1.22918475e+00,  8.94479930e-01, -1.67636171e-01,
         2.95173585e-01, -1.16299771e-01,  1.32183123e+00,
         7.54162729e-01, -3.96391690e-01, -4.77522135e-01,
         1.11174476e+00,  4.48074549e-01, -1.49217033e+00,
         2.12341905e+00, -3.59637916e-01],
       [-4.91076261e-01,  4.86555219e-01, -2.93068260e-01,
         6.27745807e-01, -7.03969240e-01,  4.24974263e-02,
        -9.98694539e-01,  6.08226836e-01, -1.32650971e+00,
         1.82342768e-01, -9.22977507e-01,  5.01198053e-01,
         4.67258155e-01,  6.16319418e-01,  1.13766396e+00,
        -3.51172239e-01, -6.28570735e-01,  1.38319182e+00,
        -1.41